In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile",
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-40929")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Lead, Global Web Analytics, Converse

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsLead, Global Web Analytics, ConverseBoston, MassachusettsBecome part of th

In [3]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Lead, Global Web Analytics, Converse",
  "experience": "5+ years of proven ability in web analytics, digital marketing, and site optimization roles leveraging data and insights to drive recommendations",
  "skills": [
    "Adobe Analytics, Google Analytics or similar web analytics tools (variables, events, segments, dimensions, custom queries, etc.)",
    "Tag management tools (e.g. GTM, DTM, etc.)",
    "Digital media, performance marketing and emerging channels such as paid search (Google ads), paid social (Facebook, Instagram ads), email, affiliates etc.",
    "Excel and PowerPoint",
    "CDP and DMP platforms (Segment, Tealium Audience Stream, Adobe Audience Manager, etc.)",
    "SQL (a plus)"
  ],
  "description": "Converse is looking for a Lead, Web analytics Global. This role is responsible for supporting the delivery of data driven solutions to improve web analytics insights, marketing performance and understanding the consumer behavior."
}
```


In [4]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead, Global Web Analytics, Converse',
 'experience': '5+ years of proven ability in web analytics, digital marketing, and site optimization roles leveraging data and insights to drive recommendations',
 'skills': ['Adobe Analytics, Google Analytics or similar web analytics tools (variables, events, segments, dimensions, custom queries, etc.)',
  'Tag management tools (e.g. GTM, DTM, etc.)',
  'Digital media, performance marketing and emerging channels such as paid search (Google ads), paid social (Facebook, Instagram ads), email, affiliates etc.',
  'Excel and PowerPoint',
  'CDP and DMP platforms (Segment, Tealium Audience Stream, Adobe Audience Manager, etc.)',
  'SQL (a plus)'],
 'description': 'Converse is looking for a Lead, Web analytics Global. This role is responsible for supporting the delivery of data driven solutions to improve web analytics insights, marketing performance and understanding the consumer behavior.'}

In [5]:
type(json_res)

dict

In [6]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [8]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [9]:
job = json_res
job['skills']

['Adobe Analytics, Google Analytics or similar web analytics tools (variables, events, segments, dimensions, custom queries, etc.)',
 'Tag management tools (e.g. GTM, DTM, etc.)',
 'Digital media, performance marketing and emerging channels such as paid search (Google ads), paid social (Facebook, Instagram ads), email, affiliates etc.',
 'Excel and PowerPoint',
 'CDP and DMP platforms (Segment, Tealium Audience Stream, Adobe Audience Manager, etc.)',
 'SQL (a plus)']

In [10]:
job

{'role': 'Lead, Global Web Analytics, Converse',
 'experience': '5+ years of proven ability in web analytics, digital marketing, and site optimization roles leveraging data and insights to drive recommendations',
 'skills': ['Adobe Analytics, Google Analytics or similar web analytics tools (variables, events, segments, dimensions, custom queries, etc.)',
  'Tag management tools (e.g. GTM, DTM, etc.)',
  'Digital media, performance marketing and emerging channels such as paid search (Google ads), paid social (Facebook, Instagram ads), email, affiliates etc.',
  'Excel and PowerPoint',
  'CDP and DMP platforms (Segment, Tealium Audience Stream, Adobe Audience Manager, etc.)',
  'SQL (a plus)'],
 'description': 'Converse is looking for a Lead, Web analytics Global. This role is responsible for supporting the delivery of data driven solutions to improve web analytics insights, marketing performance and understanding the consumer behavior.'}

In [11]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ayush, a software development engineer at McKinsey & Company. McKinsey & Company is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of McKinsey & Company 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase McKinsey & Company portfolio: {link_list}
        Remember you are Ayush, SDE at McKinsey & Company. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Data-Driven Insights with McKinsey & Company for Converse's Web Analytics Needs

Dear Hiring Manager,

I came across the job posting for Lead, Global Web Analytics at Converse, and I'm excited to introduce McKinsey & Company as a potential partner to support your web analytics needs. As a software development engineer at McKinsey & Company, I'd like to highlight our capabilities in delivering data-driven solutions that can help Converse improve web analytics insights, marketing performance, and consumer behavior understanding.

Our expertise in web analytics, digital marketing, and site optimization aligns perfectly with the requirements of the role. We have extensive experience in leveraging tools like Adobe Analytics, Google Analytics, and tag management tools (e.g., GTM, DTM) to drive recommendations and improve marketing performance. Our team is also well-versed in digital media, performance marketing, and emerging channels such as paid search, paid social, email, a